In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('transaction.csv',delimiter=",")
df.head(-5)

,payerAccountNumber,payerIdTransformed,payeeIdTransformed,originalAmount,createdAt,transaction_date,transaction_time,issuer_bank,ip_address,browser,...,bank_lost_card,channel_visa,channel_master,channel_amex,initialized_api,initialized_session,initialized_plugin,initialized_ext_api,initialized_ipg,is_fraud
0,4216895585,208,2,6110.00,2020-06-01 00:47:34,2020-06-01,00:47:34,COMMERCIAL BANK OF CEYLON PLC,Unknown,Unknown,...,0,1,0,0,0,1,0,0,0,0
1,4835598060,209,3,3274.00,2020-06-01 01:21:27,2020-06-01,01:21:27,"DFCC VARDHANA BANK, LTD.",Unknown,Unknown,...,0,1,0,0,1,0,0,0,0,0
2,4216898292,210,3,2303.00,2020-06-01 01:21:30,2020-06-01,01:21:30,COMMERCIAL BANK OF CEYLON PLC,Unknown,Unknown,...,1,1,0,0,1,0,0,0,0,0
3,4216892908,211,3,9000.00,2020-06-01 01:21:32,2020-06-01,01:21:32,COMMERCIAL BANK OF CEYLON PLC,Unknown,Unknown,...,0,1,0,0,1,0,0,0,0,0
4,4216899313,212,3,6250.00,2020-06-01 01:21:34,2020-06-01,01:21:34,COMMERCIAL BANK OF CEYLON PLC,Unknown,Unknown,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256284,5391386808,27530,5,9357.62,2021-04-26 13:58:14,2021-04-26,13:58:14,PEOPLE'S BANK,Unknown,Unknown,...,0,0,1,0,0,0,0,1,0,0
256285,4216895158,27612,5,6366.34,2021-04-26 13:58:29,2021-04-26,13:58:29,COMMERCIAL BANK OF CEYLON PLC,Unknown,Unknown,...,0,1,0,0,0,0,0,1,0,0
256286,4216896969,27680,5,6579.78,2021-04-26 13:58:36,2021-04-26,13:58:36,COMMERCIAL BANK OF CEYLON PLC,Unknown,Unknown,...,0,1,0,0,0,0,0,1,0,0
256287,5381477516,27779,5,8797.33,2021-04-26 13:58:42,2021-04-26,13:58:42,SAMPATH BANK PLC,Unknown,Unknown,...,0,0,1,0,0,0,0,1,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256294 entries, 0 to 256293
Data columns (total 34 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   payerAccountNumber            256294 non-null  int64  
 1   payerIdTransformed            256294 non-null  int64  
 2   payeeIdTransformed            256294 non-null  int64  
 3   originalAmount                256294 non-null  float64
 4   createdAt                     256294 non-null  object 
 5   transaction_date              256294 non-null  object 
 6   transaction_time              256294 non-null  object 
 7   issuer_bank                   256294 non-null  object 
 8   ip_address                    256294 non-null  object 
 9   browser                       256294 non-null  object 
 10  funding_method                256294 non-null  int64  
 11  secure_3ds                    256294 non-null  int64  
 12  success_3ds                   256294 non-nul

In [4]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [5]:
df = MultiColumnLabelEncoder(columns = ['issuer_bank','ip_address','browser']).fit_transform(df)

In [6]:
df.to_csv('transaction_preprocess.csv')

In [7]:
df['createdAt']=pd.to_datetime(df.createdAt)
df['createdAt'] = df.createdAt.values.astype(np.int64)
df['transaction_date']=pd.to_datetime(df.transaction_date)
df['transaction_time']=pd.to_datetime(df.transaction_time)
df['transaction_date'] = df['transaction_date'].apply(lambda x: x.value)
df['transaction_time'] = df['transaction_time'].apply(lambda x: x.value)
con_vars = ['payerAccountNumber','payerIdTransformed', 'payeeIdTransformed','createdAt','transaction_date','transaction_time',
           'issuer_bank','ip_address','browser']
df.fillna(0)
scaler = StandardScaler()
df[con_vars]=scaler.fit_transform(df[con_vars])
df[con_vars]  =round(df[con_vars],10)
df.to_csv('standerdize.csv',index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256294 entries, 0 to 256293
Data columns (total 34 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   payerAccountNumber            256294 non-null  float64
 1   payerIdTransformed            256294 non-null  float64
 2   payeeIdTransformed            256294 non-null  float64
 3   originalAmount                256294 non-null  float64
 4   createdAt                     256294 non-null  float64
 5   transaction_date              256294 non-null  float64
 6   transaction_time              256294 non-null  float64
 7   issuer_bank                   256294 non-null  float64
 8   ip_address                    256294 non-null  float64
 9   browser                       256294 non-null  float64
 10  funding_method                256294 non-null  int64  
 11  secure_3ds                    256294 non-null  int64  
 12  success_3ds                   256294 non-nul